In [71]:
# make sure to install python rediit wrapper
#!pip install praw 

In [72]:
import os 
import openai
import pprint
import praw
pp = pprint.PrettyPrinter(depth=10)

In [73]:
openai.api_key = os.environ["OPENAI_KEY"]

In [75]:
# create a new user app at reddit for an api key to access posts. 
# https://old.reddit.com/prefs/apps/
reddit_key = os.environ.get("REDDIT_KEY", "") 
reddit_secret = os.environ.get("REDDIT_SECRET", "")

In [68]:
def ask_open_ai(prompt):
    res = openai.Completion.create(model="text-davinci-003",
                                   prompt = prompt,
                                   temperature = 0,
                                   top_p = 1.0,
                                   max_tokens = 256)
    return res


def get_titles_and_comments(sub="stocks", limit=10, comment_limit=5, skip=2):
    subreddit = reddit.subreddit(sub)
    content = {}
    for counter, post in enumerate(subreddit.hot(limit=limit)):
        if counter < skip:
            continue
        counter += (1 - skip)
        content[counter] = ""
        content[counter] += f"Title: {post.title}\n\nComments:\n\n"
        comment_count = 0
        
        for comment in reddit.submission(post.id).comments:
            if '[deleted]' not in comment.body and comment.body.strip() != "":
                content[counter] += f"{comment.body}\n"
                comment_count += 1
            if comment_count > comment_limit:
                print(f" subject #  {counter} done: {post.title} with {comment_count} comments" )
                break
    
    return content

def create_prompt(contents):
    task = "Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative or neutral. If no ticker or company is mentioned write 'No company mentioned'\n\n"
    return f"{task}{contents}"   

def get_sentiment_report(data):
    report = {}
    for k,v in data.items():
        resp = ask_open_ai(create_prompt(v))
        report[k] = resp['choices'][0]['text']
    return report



In [66]:
data = get_titles_and_comments('stocks')

 subject #  1 done: Lost 90k gambling, don't make the same mistakes as me with 6 comments
 subject #  2 done: Massive Misconception About the Writers Strike with 6 comments
 subject #  3 done: Why do so many people here spend time picking stocks that are already a huge portion of the S&P? with 6 comments
 subject #  5 done: Has disney bottomed? with 6 comments
 subject #  7 done: How bad is the bankruptcy from Evergrande for the stockmarket? with 6 comments


In [69]:
sentiment_report = get_sentiment_report(data)

In [70]:
for k,v in sentiment_report.items():
    print( v )


No company mentioned.

No company mentioned.

No company mentioned.

No company mentioned.

Company: Disney (DIS)
Sentiment: Negative

No company mentioned.

Company: Evergrande
Sentiment: Negative

No company mentioned
